### Assignment-03

- Formulate three original hypotheses based on the raw US Accidents dataset, and perform appropriate statistical hypothesis tests to validate each one.

In [3]:
import numpy as np
import pandas as pd
df=pd.read_csv("US_Accidents_March23.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [4]:
def permutation_test(x, y, n_permutations=1000, alternative="greater", seed=42):
    np.random.seed(seed)

    x = np.array(x)
    y = np.array(y)

    observed_diff = x.mean() - y.mean()
    combined = np.concatenate([x, y])

    perm_diffs = []
    for _ in range(n_permutations):
        np.random.shuffle(combined)
        perm_x = combined[:len(x)]
        perm_y = combined[len(x):]
        perm_diffs.append(perm_x.mean() - perm_y.mean())

    perm_diffs = np.array(perm_diffs)

    if alternative == "greater":
        p_value = np.mean(perm_diffs >= observed_diff)
    elif alternative == "less":
        p_value = np.mean(perm_diffs <= observed_diff)
    else:
        p_value = np.mean(np.abs(perm_diffs) >= abs(observed_diff))

    return observed_diff, p_value


Hypothesis 1: 
Nighttime accidents are more severe than daytime accidents

In [10]:
day = df[df["Sunrise_Sunset"] == "Day"]["Severity"].dropna()
night = df[df["Sunrise_Sunset"] == "Night"]["Severity"].dropna()
diff, p_value = permutation_test(
    night, day,
    n_permutations=100,
    alternative="greater"
)
print("Mean severity difference (Night - Day):", diff)
print("p-value:", p_value)
print(
    "Hypothesis supported" if p_value < 0.05
    else "Hypothesis not supported"
)

Mean severity difference (Night - Day): 0.009673063529520132
p-value: 0.0
Hypothesis supported


Hypothesis 2: 
Accidents near traffic signals have higher severity

In [11]:
signal = df[df["Traffic_Signal"] == True]["Severity"].dropna()
no_signal = df[df["Traffic_Signal"] == False]["Severity"].dropna()
diff, p_value = permutation_test(
    signal, no_signal,
    n_permutations=100,
    alternative="greater"
)

print("Mean severity difference (Signal - No Signal):", diff)
print("p-value:", p_value)
print(
    "Hypothesis supported" if p_value < 0.05
    else "Hypothesis not supported"
)

Mean severity difference (Signal - No Signal): -0.1440243048909231
p-value: 1.0
Hypothesis not supported


Hypothesis 3: 
Low visibility increases accident severity

In [12]:
low_vis = df[df["Visibility(mi)"] < 3]["Severity"].dropna()
high_vis = df[df["Visibility(mi)"] >= 3]["Severity"].dropna()
diff, p_value = permutation_test(
    low_vis, high_vis,
    n_permutations=100,
    alternative="greater"
)

print("Mean severity difference (Low - High visibility):", diff)
print("p-value:", p_value)
print(
    "Hypothesis supported" if p_value < 0.05
    else "Hypothesis not supported"
)

Mean severity difference (Low - High visibility): 0.01439966021159833
p-value: 0.0
Hypothesis supported


# Task
Analyze the US Accidents dataset by loading and exploring it, then formulate and statistically test three hypotheses: one on the impact of weather conditions on accident severity/frequency, one on the impact of time of day on accident frequency/severity, and one on the influence of road features on accident occurrence/severity. Finally, summarize and interpret the results of these tests.

## Load and Explore Data

### Subtask:
Load the US Accidents dataset and perform initial exploration to understand its structure, columns, and potential areas for hypothesis formulation. This includes displaying head, info, and descriptive statistics.


**Reasoning**:
First, I need to import the pandas library to work with DataFrames. Then, I will load the 'US_Accidents_Dec21_updated.csv' dataset into a pandas DataFrame named `df` as per the instructions. After loading, I will display the first 5 rows, print a concise summary of the DataFrame (info), and generate descriptive statistics to fulfill the subtask requirements.



In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('US_Accidents_Dec21_updated.csv')

# Display the first 5 rows
print("First 5 rows of the DataFrame:")
print(df.head())

# Print a concise summary of the DataFrame
print("\nDataFrame Information:")
df.info()

# Generate descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())

FileNotFoundError: [Errno 2] No such file or directory: 'US_Accidents_Dec21_updated.csv'

**Reasoning**:
The previous code failed because the specified CSV file was not found. I need to update the file path to the correct location where the dataset is stored, which is typically in the `/kaggle/input/` directory in a Colab environment for a Kaggle dataset.



In [ ]:
import pandas as pd

# Load the dataset with the corrected path
df = pd.read_csv('/kaggle/input/us-accidents-dec21-updated/US_Accidents_Dec21_updated.csv')

# Display the first 5 rows
print("First 5 rows of the DataFrame:")
print(df.head())

# Print a concise summary of the DataFrame
print("\nDataFrame Information:")
df.info()

# Generate descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/us-accidents-dec21-updated/US_Accidents_Dec21_updated.csv'

**Reasoning**:
The previous attempts to load the dataset failed due to a FileNotFoundError, indicating the dataset was not present in the specified paths. To resolve this in a Colab environment for a Kaggle dataset, I will add steps to install the Kaggle API client, download the dataset from Kaggle, and then unzip it into the working directory before loading it with pandas.



In [ ]:
import pandas as pd

# Install Kaggle API client if not already installed
!pip install kaggle --quiet

# --- IMPORTANT: Manual step for Kaggle API key --- #
# For this to work, you need to upload your kaggle.json file.
# Steps:
# 1. Go to Kaggle, click on your profile picture -> 'Account'.
# 2. Scroll down to 'API' section and click 'Create New API Token'. This downloads 'kaggle.json'.
# 3. In Colab, run the following (if not already done):
#    from google.colab import files
#    files.upload() # Then upload kaggle.json
# 4. Move kaggle.json to the correct directory and set permissions:
#    !mkdir -p ~/.kaggle
#    !mv kaggle.json ~/.kaggle/
#    !chmod 600 ~/.kaggle/kaggle.json
# --- End of manual step instructions --- #

# Define the Kaggle dataset details
kaggle_dataset_owner = 'sourabhshah1'
kaggle_dataset_name = 'us-accidents-dec21-updated'
zip_file_name = f'{kaggle_dataset_name}.zip'
csv_file_name = 'US_Accidents_Dec21_updated.csv'

# Download the dataset from Kaggle
print(f"Downloading {kaggle_dataset_owner}/{kaggle_dataset_name} dataset...")
!kaggle datasets download -d {kaggle_dataset_owner}/{kaggle_dataset_name}

# Unzip the downloaded file. The -o flag overwrites if already exists.
print(f"Unzipping {zip_file_name}...")
!unzip -o {zip_file_name} -d .

# Load the dataset from the unzipped CSV file (now in the current directory)
df = pd.read_csv(csv_file_name)

# Display the first 5 rows
print("\nFirst 5 rows of the DataFrame:")
print(df.head())

# Print a concise summary of the DataFrame
print("\nDataFrame Information:")
df.info()

# Generate descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Reasoning**:
The previous code failed because the Kaggle API key was not properly configured, leading to a `KeyError: 'username'` during the dataset download attempt, and subsequently a `FileNotFoundError` for the zip and CSV files. The core issue is the manual step of uploading and configuring `kaggle.json`. I will re-provide the same code block, as its logic is correct for downloading a Kaggle dataset, but add a prominent print statement to re-emphasize the critical manual steps required to set up the Kaggle API key before executing the download command.



In [ ]:
import pandas as pd

# Install Kaggle API client if not already installed
!pip install kaggle --quiet

# --- IMPORTANT: MANUAL KAGGLE API KEY SETUP REQUIRED --- #
# YOU MUST UPLOAD YOUR KAGGLE.JSON AND CONFIGURE IT.
# Please ensure you have completed the following steps before proceeding:
# 1. Go to Kaggle, click on your profile picture -> 'Account'.
# 2. Scroll down to 'API' section and click 'Create New API Token'. This downloads 'kaggle.json'.
# 3. In Colab, run the following commands (if not already done) to upload and configure your API key:
#    from google.colab import files
#    files.upload() # Then upload kaggle.json
#    !mkdir -p ~/.kaggle
#    !mv kaggle.json ~/.kaggle/
#    !chmod 600 ~/.kaggle/kaggle.json
# Without these steps, the dataset download will fail.
print("\n*** ENSURE KAGGLE API KEY (kaggle.json) IS UPLOADED AND CONFIGURED AS PER INSTRUCTIONS ABOVE ***\n")
# --- END OF MANUAL STEP INSTRUCTIONS --- #

# Define the Kaggle dataset details
kaggle_dataset_owner = 'sourabhshah1'
kaggle_dataset_name = 'us-accidents-dec21-updated'
zip_file_name = f'{kaggle_dataset_name}.zip'
csv_file_name = 'US_Accidents_Dec21_updated.csv'

# Download the dataset from Kaggle
print(f"Downloading {kaggle_dataset_owner}/{kaggle_dataset_name} dataset...")
!kaggle datasets download -d {kaggle_dataset_owner}/{kaggle_dataset_name}

# Unzip the downloaded file. The -o flag overwrites if already exists.
print(f"Unzipping {zip_file_name}...")
!unzip -o {zip_file_name} -d .

# Load the dataset from the unzipped CSV file (now in the current directory)
df = pd.read_csv(csv_file_name)

# Display the first 5 rows
print("\nFirst 5 rows of the DataFrame:")
print(df.head())

# Print a concise summary of the DataFrame
print("\nDataFrame Information:")
df.info()

# Generate descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())


*** ENSURE KAGGLE API KEY (kaggle.json) IS UPLOADED AND CONFIGURED AS PER INSTRUCTIONS ABOVE ***

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
Unzipping us-accidents-dec21-updated.zip...
unzip:  cannot find or open us-accidents-dec21-updated.zip, us-accidents-dec21-updated.zip.zip or us-accidents-dec21-updated.zip.ZIP.


FileNotFoundError: [Errno 2] No such file or directory: 'US_Accidents_Dec21_updated.csv'

**Reasoning**:
The previous code failed because the Kaggle API key was not properly configured, leading to a `KeyError: 'username'` during the dataset download attempt. The critical manual steps for uploading and configuring `kaggle.json` were present as comments but not actively executed. To fix this, I will incorporate the necessary Colab-specific commands to handle the `kaggle.json` upload and configuration directly into the code block, preceding the dataset download, along with clear user instructions.



In [ ]:
import pandas as pd
from google.colab import files # Import files for uploading

# Install Kaggle API client if not already installed
!pip install kaggle --quiet

# --- IMPORTANT: KAGGLE API KEY SETUP --- #
# YOU MUST UPLOAD YOUR KAGGLE.JSON AND CONFIGURE IT.
# This part requires user interaction in the Colab environment.
print("\n*** PLEASE FOLLOW THE PROMPT BELOW TO UPLOAD YOUR KAGGLE.JSON FILE ***")
print("Go to Kaggle, click on your profile picture -> 'Account'.")
print("Scroll down to 'API' section and click 'Create New API Token'. This downloads 'kaggle.json'.")
print("Then, when prompted by the 'files.upload()' dialog, select your downloaded 'kaggle.json'.\n")

# Upload kaggle.json
files.upload() # This will open a file selection dialog

# Move kaggle.json to the correct directory and set permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\n*** Kaggle API key configured successfully! ***\n")
# --- END OF KAGGLE API KEY SETUP --- #

# Define the Kaggle dataset details
kaggle_dataset_owner = 'sourabhshah1'
kaggle_dataset_name = 'us-accidents-dec21-updated'
zip_file_name = f'{kaggle_dataset_name}.zip'
csv_file_name = 'US_Accidents_Dec21_updated.csv'

# Download the dataset from Kaggle
print(f"Downloading {kaggle_dataset_owner}/{kaggle_dataset_name} dataset...")
!kaggle datasets download -d {kaggle_dataset_owner}/{kaggle_dataset_name}

# Unzip the downloaded file. The -o flag overwrites if already exists.
print(f"Unzipping {zip_file_name}...")
!unzip -o {zip_file_name} -d .

# Load the dataset from the unzipped CSV file (now in the current directory)
df = pd.read_csv(csv_file_name)

# Display the first 5 rows
print("\nFirst 5 rows of the DataFrame:")
print(df.head())

# Print a concise summary of the DataFrame
print("\nDataFrame Information:")
df.info()

# Generate descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())